In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
cd '/home/dfried/projects/onecommon/aaai2020/experiments'

/home/dfried/projects/onecommon/aaai2020/experiments


In [3]:
import models
import data
import utils
from domain import get_domain
from corpora.reference_sentence import ReferenceSentenceCorpus

In [4]:
ls expts/rel3_tsel_ref_dial_model_separate/

hierarchical_only-self_gb-this-mentioned_mask-pad_h2o-1.out
hierarchical_only-self_gb-this-mentioned_mask-pad_h2o-1_sigmoid.out
hierarchical_only-self_mark-dots-mentioned_mask-pad_h2o-1.out
hierarchical_only-self_mark-dots-mentioned_mask-pad_h2o-1_sigmoid.out
hierarchical_only-self_mask-pad_h2o-1.out
hierarchical_only-self_mbfix-cumulative-mentioned_next-mention_mask-pad_h2o-1.out
hierarchical_only-self_mbfix-cumulative-mentioned_next-mention_mask-pad_h2o-1_sigmoid.out
hierarchical_only-self_mbfix-cumulative-partner-mentioned_next-mention_mask-pad_h2o-1.out
hierarchical_only-self_mbfix-cumulative-partner-mentioned_next-mention_mask-pad_h2o-1_sigmoid.out
hierarchical_only-self_mbfix-last_mentioned_next-mention_mask-pad_h2o-1.out
hierarchical_only-self_mbfix-last_mentioned_next-mention_mask-pad_h2o-1_sigmoid.out
hierarchical_only-self_mbfix-last_mentioned+this-partner-mentioned+cumulative-mentioned+cumulative-partner-mentioned_next-mention_mask-pad_h2o-1.out
hierarchical_only-self_mbfix-

In [5]:
domain = get_domain('one_common')
unk_threshold = 10
model_ty = models.get_model_type('rnn_reference_model')

ctx_encoder_type = 'rel_attn_encoder_3'

# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_mask-pad_max-pool_rel-2_feed-context-attend_no-word-attention_ablate-properties_1_best.th'

model_fnames = {
    'basic': 'rel3_tsel_ref_dial_model_separate/hierarchical_only-self_mask-pad_h2o-1_1_best.th',
    'mark_dots_mentioned': 'rel3_tsel_ref_dial_model_separate/hierarchical_only-self_mark-dots-mentioned_mask-pad_h2o-1_1_best.th'
}

In [6]:
seed = 1
corpus = ReferenceSentenceCorpus(domain, 'data/onecommon',
                            train='train_reference_{}.txt'.format(seed),
                            valid='valid_reference_{}.txt'.format(seed),
                            test='test_reference_{}.txt'.format(seed),
                            freq_cutoff=unk_threshold, verbose=True)

dataset data/onecommon/train_reference_1.txt, total 545056, unks 4302, ratio 0.79%
dataset data/onecommon/valid_reference_1.txt, total 69348, unks 686, ratio 0.99%
dataset data/onecommon/test_reference_1.txt, total 67284, unks 636, ratio 0.95%


In [7]:
def load_json(file):
    import json
    with open(file, 'r') as f:
        return json.load(f)

In [8]:
scenarios1 = load_json('data/onecommon/aaai_train_scenarios.json')
scenarios2 = load_json('data/onecommon/aaai_train_scenarios_2.json')

In [9]:
len(scenarios1)

9000

In [10]:
len(scenarios2)

4500

In [11]:
scenarios1[0].keys()

dict_keys(['kbs', 'shared', 'uuid'])

In [12]:
train_scenarios = {
    scenario['uuid']: scenario
    for scenario in scenarios1
}

In [13]:
hit_count = 0
miss_count = 0
for scenario in scenarios2:
    uuid = scenario['uuid']
    if uuid in train_scenarios:
        assert train_scenarios[uuid] == scenario
        hit_count += 1
    else:
        miss_count += 1

In [14]:
print("hits: {}".format(hit_count))
print("misses: {}".format(miss_count))

hits: 0
misses: 4500


In [15]:
corpus

In [17]:
models = {
    name: utils.load_model(model_fname, map_location='cpu')
    for name, model_fname in model_fnames.items()
}

/home/dfried/miniconda3/envs/onecommon/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'models.rnn_reference_model.HierarchicalRnnReferenceModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [19]:
batchsize=1
validset, validset_stats = corpus.valid_dataset(batchsize)

In [20]:
validset_stats

{'n': 74030, 'self_n': 37015, 'nonpadn': 69348, 'self_nonpadn': 34674}

In [21]:
import torch

In [22]:
from agent import RnnAgent

In [23]:
from selfplay import make_parser

In [24]:
parser = make_parser()
args = parser.parse_args('--temperature 0.25'.split())
# args = parser.parse_args('--temperature 0.01'.split())

In [25]:
def is_selection(out):
    return '<selection>' in out

In [26]:
# display

In [27]:
from dialog import DialogLogger

In [28]:
from IPython.display import SVG, display, HTML

In [29]:
def display_svgs(svgs):
    no_wrap_div = '<div style="white-space: nowrap">' + ''.join(svgs) + '</div>'
    display(HTML(no_wrap_div))

In [30]:
def display_attn(scenario, attn, agent_id, name=None):
    attn = attn.flatten().detach().numpy()
    if name is not None:
        print("{}: {}".format(name, attn))
    display_svgs([DialogLogger._scenario_to_svg(scenario)[agent_id], DialogLogger._attention_to_svg(scenario, agent_id, attn)])

In [31]:
pdb on

Automatic pdb calling has been turned ON


In [32]:
# training emulation

In [33]:
trainset, trainset_stats = corpus.train_dataset(bsz=1)

In [34]:
trainset[0].scenario_ids[0] in train_scenarios

True

In [35]:
# ctx: this player's dots
# inpt / tgt: dialogue word indices (tgt are shifted by 1)
# ref_inpt

In [50]:
for ix in range(len(trainset)):
#     ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
#     [scenario_id], real_ids, agents, chat_ids, sel_idxs, \
#     lens, rev_idxs, hid_idxs, all_num_markables = trainset[ix]
    
    ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
    [scenario_id], real_ids, partner_real_ids, agents, chat_ids, sel_idxs, \
    lens, rev_idxs, hid_idxs, num_markables, \
    is_self, partner_ref_inpts, partner_ref_tgts_our_view, all_partner_num_markables = trainset[ix]

    if 'C_d0de3d0144df405e998ae718539ff9a4' in chat_ids:
#     if scenario_id == 'S_UoujG1GG6KscmnbN':
        print("found it")
        scenario = train_scenarios[scenario_id]
        break

found it


In [47]:
model = models['mark_dots_mentioned']

In [48]:
alice = RnnAgent(model, args)
bob = RnnAgent(model, args)

In [51]:
# training emulation
scenario = train_scenarios[scenario_id]
agent = alice
agent.feed_context(ctx.flatten())
agent.real_ids = real_ids
agent.agent_id = agents[0]
    
samples = 10
    
writer = alice
conv = []
speaker = []
words_left = 5000

sentence_ix = 0

assert len(inpts) == len(tgts)
    
while sentence_ix < len(inpts):
    words_og = [corpus.word_dict.idx2word[ix] for ix in inpts[sentence_ix].flatten().numpy()]
#     for _ in range(samples):
#         pred_outs, _, _, _, extra = writer.model.write(writer.ctx_h, writer.lang_h, words_left, writer.args.temperature)
#         print('{}\t{}'.format(writer.agent_id, ' '.join(writer._decode(outs.narrow(0,1,outs.size(0)-1), writer.model.word_dict))))
    print(' '.join(words_og))
    if ref_inpts[sentence_ix] is not None:
        dots_mentioned = (ref_tgts[sentence_ix].sum(dim=1) > 0)
        display_attn(scenario, dots_mentioned.float(), writer.agent_id, name='dots_mentioned')
    else:
        dots_mentioned = None
    out = writer.write(
        max_words=words_left,
        force_words=[words_og],
        start_token=words_og[0],
        dots_mentioned=dots_mentioned
    )
#     out = writer.write(max_words=words_left)
    
    if writer.extras[-1]['feed_ctx_attn_prob'] is not None:
        display_attn(scenario, writer.extras[-1]['feed_ctx_attn_prob'], writer.agent_id, name='feed attention')
    
    if 'word_ctx_attn_prob_mean' in writer.extras[-1]:
        display_attn(scenario, writer.extras[-1]['word_ctx_attn_prob_mean'], writer.agent_id, name='word attention mean')
        
    for ix, ix_attn in enumerate(writer.extras[-1]['word_ctx_attn_probs']):
        display_attn(scenario, ix_attn, writer.agent_id, name='t {}: {}'.format(ix, out[ix]))
        
    print('{}\t{}'.format(writer.agent_id, ' '.join(out)))
    print()
    words_left -= len(out)
    conv.append(out)
    speaker.append(writer.agent_id)
#     reader.read(out)
    if is_selection(out) or words_left <= 1:
        break
#     writer, reader = reader, writer
    sentence_ix += 1

YOU: i see a dark grey dot with a slightly smaller darker dot below it <eos>
dots_mentioned: [1. 1. 0. 0. 0. 0. 0.]


word attention mean: [0.44916221 0.30394575 0.00901062 0.00566747 0.0950246  0.06985968
 0.06732957]


t 0: i: [5.1563394e-01 4.6030292e-01 2.0864936e-06 1.4721269e-06 2.2387674e-02
 4.4005289e-04 1.2317613e-03]


t 1: see: [0.19833878 0.20594174 0.02070414 0.01583051 0.18872908 0.18491638
 0.18553942]


t 2: a: [4.77102369e-01 4.98029530e-01 1.05115824e-07 1.21620360e-07
 2.41233595e-02 7.87135214e-05 6.65728177e-04]


...

t 13: it: [0.18968911 0.18664761 0.02898193 0.02145691 0.18887568 0.19220763
 0.1921411 ]


t 14: <eos>: [2.0278631e-01 1.8740644e-01 2.4202293e-04 1.1126054e-04 2.1638253e-01
 1.8699157e-01 2.0607992e-01]


1	i see a dark grey dot with a slightly smaller darker dot below it <eos>

THEM: i think i have that too . above the larger one and to its left i have a much smaller medium gray dot <eos>


AssertionError: 

> /home/dfried/projects/onecommon/aaai2020/experiments/models/rnn_reference_model.py(614)_language_conditioned_dot_attention()
    612         if vars(self.args).get('mark_dots_mentioned', False):
    613             # TODO: consider tiling this indicator feature across more dimensions
--> 614             assert dots_mentioned is not None
    615             ctx_h_marked = torch.cat((ctx_h, dots_mentioned.float().unsqueeze(-1)), -1)
    616         else:

ipdb> q


In [52]:
import json
from utils import ContextGenerator

In [53]:
ctx_gen = ContextGenerator('data/onecommon/shared_4.txt')

In [54]:
with open('data/onecommon/shared_4.json', "r") as f:
    scenario_list = json.load(f)
selfplay_scenarios = {scenario['uuid']: scenario for scenario in scenario_list}

In [55]:
scenario = None
for ix, ([scenario_id], (ctx1, ctx2), (ids1, ids2)) in enumerate(ctx_gen.iter()):
#     if scenario_id == 'S_11J8n40u4HRVduwi':
#         print('found it')
#         scenario = scenarios[scenario_id]
#         break
    if ix == 2:
        scenario = selfplay_scenarios[scenario_id]
        print(scenario_id)
        break

S_ZsxpAYY2FtpXzUHJ


In [ ]:
# selfplay emulation
scenario = selfplay_scenarios[scenario_id]
for agent, agent_id, ctx, real_ids in zip([alice, bob], [0, 1], [ctx1, ctx2], [ids1, ids2]):
    agent.feed_context(ctx)
    agent.real_ids = real_ids
    agent.agent_id = agent_id
    
dot_mention_samples = 1
utt_samples = 10
    
writer, reader = [alice, bob]
conv = []
speaker = []
words_left = 5000

num_dots_to_mention = 2

temperature_override = 0.005

def make_dots_mentioned():
#     # fixed
#     return torch.BoolTensor([[1,0,0,0,0,0,0]])

    # random 
    mentioned = torch.zeros(1,7).bool()
    for _ in range(num_dots_to_mention):
        ix = torch.randint(7, (1,))
        mentioned[:,ix] = 1
    return mentioned

display_context = True

while True:
    for dm in range(dot_mention_samples):
        dots_mentioned = make_dots_mentioned()
        if display_context:
            display_attn(scenario, dots_mentioned.float(), writer.agent_id, name='dots_mentioned')
        for us in range(utt_samples):
            outs, _, _, _, extra = writer.model.write(writer.ctx_differences, writer.ctx_h, writer.lang_h, words_left, temperature_override, dots_mentioned=dots_mentioned)
            print('{}\t{}'.format(writer.agent_id, ' '.join(writer._decode(outs.narrow(0,1,outs.size(0)-1), writer.model.word_dict))))
        print()
        
    dots_mentioned = make_dots_mentioned()
    if display_context:
        display_attn(scenario, dots_mentioned.float(), writer.agent_id, name='dots_mentioned')
    out = writer.write(max_words=words_left, dots_mentioned=dots_mentioned, temperature_override=temperature_override)
    print('{}\t{}'.format(writer.agent_id, ' '.join(out)))
    print()
    
    if display_context and writer.extras[-1].get('feed_ctx_attn_prob', None) is not None:
        display_attn(scenario, writer.extras[-1]['feed_ctx_attn_prob'], writer.agent_id, name='feed attention')
    
    if display_context and writer.extras[-1].get('word_ctx_attn_prob_mean', None) is not None:
        display_attn(scenario, writer.extras[-1]['word_ctx_attn_prob_mean'], writer.agent_id, name='word attention mean')
        
#     for ix, ix_attn in enumerate(writer.extras[-1]['dot_attention2s']):
#         display_attn(scenario, ix_attn, writer.agent_id, name='t {}: {}'.format(ix, out[ix]))
    print('{}\t{}'.format(writer.agent_id, ' '.join(out)))
    print('-'*60)
        
    words_left -= len(out)
    conv.append(out)
    speaker.append(writer.agent_id)
    reader.read(out, dots_mentioned=dots_mentioned)
    if is_selection(out) or words_left <= 1:
        break
    writer, reader = reader, writer